In [1]:
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import torch

In [2]:
import transformers
transformers.__version__

'4.39.0.dev0'

In [3]:
starcoder_7b = "bigcode/starcoder2-7b"

In [4]:
coder_tokenizer = AutoTokenizer.from_pretrained(starcoder_7b)

In [5]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                  bnb_4bit_quant_type='nf4')

In [6]:
coder_model = AutoModelForCausalLM.from_pretrained(starcoder_7b,
                                                   quantization_config=quant_config,
                                                  device_map="auto",
                                                  resume_download=True)

model.safetensors.index.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
def generate_output(text, tokens):
    tokenized = coder_tokenizer(text, return_tensors='pt')
    logits = coder_model.generate(**tokenized.to(device),
                                  max_new_tokens=tokens,
                                 temperature=0.5,
                                 do_sample=True)
    return coder_tokenizer.decode(logits[0])
    

In [21]:
test_stmt = "provide the code for connecting youtube API via python"
gen_output = generate_output(test_stmt, 500)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


provide the code for connecting youtube API via python

# + id="4U04aE1K4q2v"
from google.colab import drive
drive.mount('/content/drive')

# + id="r32s86114q2w"
#!pip install google-api-python-client

# + id="c5n6h34z4q2w"
import os
import json
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# + id="5q2-d1l34q2w"
api_service_name = 'youtube'
api_version = 'v3'
client_secrets_file = '/content/drive/My Drive/Data/youtube_api/client_secret.json'

# + id="dC6h_g_K4q2w"
# Get credentials and create an API client
scopes = ['https://www.googleapis.com/auth/youtube.readonly']
flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
credentials = flow.run_console()
youtube = build(api_service_name, api_version, credentials = credentials)

# + id="o8t6g_Y44q2w"
# To retrieve the channel ID, run the following command:
# $ youtube-dl --get-id https://www.youtube.com/watch?v=x3U10E